In [4]:
import os
import subprocess
import pkg_resources

def clone_repo(github_url):
    """
    Clone the GitHub repository and check if requirements.txt exists.
    """
    repo_name = github_url.split('/')[-1]
    if os.path.exists(repo_name):
        print(f"Repository '{repo_name}' already exists. Skipping clone.")
    else:
        subprocess.run(['git', 'clone', github_url])
        print(f"Cloned repository '{repo_name}'.")

    req_file_path = os.path.join(repo_name, 'requirements.txt')
    if os.path.exists(req_file_path):
        print("Found requirements.txt file.")
        return req_file_path
    else:
        print("No requirements.txt file found in the repository.")
        return None

def check_compatibility(req_file_path, report_file="mismatch_report.txt"):
    """
    Check compatibility of each library in requirements.txt with the installed libraries,
    and save mismatches in a report file.
    """
    incompatible_packages = []

    # Read the requirements file
    with open(req_file_path, 'r') as req_file:
        for line in req_file:
            if line.strip():
                try:
                    # Parse the package and version from the line
                    req = pkg_resources.Requirement.parse(line.strip())
                    pkg_name = req.name
                    req_version = req.specs[0][1] if req.specs else None

                    # Check if package is installed
                    installed_version = pkg_resources.get_distribution(pkg_name).version

                    # Compare versions
                    if req_version and installed_version != req_version:
                        print(f"Mismatch for {pkg_name}: Required {req_version}, Installed {installed_version}")
                        incompatible_packages.append((pkg_name, req_version, installed_version))
                    else:
                        print(f"{pkg_name} is compatible (version {installed_version}).")
                except pkg_resources.DistributionNotFound:
                    print(f"{pkg_name} is not installed.")
                    incompatible_packages.append((pkg_name, req_version, "Not Installed"))
                except Exception as e:
                    print(f"Error checking {pkg_name}: {e}")

    # Save incompatible packages to a report file
    if incompatible_packages:
        with open(report_file, 'w') as f:
            f.write("Incompatible or missing packages:\n")
            for pkg_name, req_version, installed_version in incompatible_packages:
                f.write(f"{pkg_name}: Required {req_version}, Installed {installed_version}\n")
        print(f"\nMismatch report saved to {report_file}")
    else:
        print("All packages are compatible.")

# GitHub URL of the repo to clone
github_url = "https://github.com/"  # Replace with actual repo URL

# Clone the repo and check for requirements
req_file_path = clone_repo(github_url)
if req_file_path:
    check_compatibility(req_file_path)
else:
    print("No requirements.txt to verify.")


Repository 'pypi-notifier' already exists. Skipping clone.
Found requirements.txt file.
Mismatch for Flask: Required 2.3.3, Installed 2.2.5
Flask-Caching is not installed.
Flask-SQLAlchemy is not installed.
GitHub-Flask is not installed.
psycopg2-binary is not installed.
pystmark is not installed.
Mismatch for pytest: Required 7.1.2, Installed 7.4.4
raven is not installed.
redis is not installed.
Mismatch for requests: Required 2.28.1, Installed 2.32.3
Mismatch for SQLAlchemy: Required 1.4.40, Installed 2.0.36
verlib is not installed.
Mismatch for blinker: Required 1.6.2, Installed 1.4
zappa is not installed.
setuptools is compatible (version 75.1.0).

Mismatch report saved to mismatch_report.txt
